In [1]:
import Data_Load
import Preprocess as prpr

from parameters import *

In [2]:
# import dlp

In [3]:
import pandas as pd
import numpy as np
import pickle

In [5]:
!pip install pycaret

Defaulting to user installation because normal site-packages is not writeable


In [6]:
!pip install cupy-cuda100

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 369.7 MB 16 kB/s  eta 0:00:01


In [7]:
from pycaret.regression import *

## 1. 기본 데이터 
#### (1시간 단위의 교통 카드 이력 데이터, 정류장별 위/경도 좌표 포함)

In [5]:
base_data = pd.read_pickle("/home/seho/Passenger_Demand/data/base_data.pkl")
target_col = "totalcnt"

In [6]:
base_data.to_csv("/home/seho/Passenger_Demand/data/base_data.csv")

In [8]:
base_data = pd.read_csv("/project/work/Passenger_Demand/data/base_data.csv")
target_col = "totalcnt"

In [9]:
base_data["transdate"][0]

'2020-04-08 05:00:00'

## 2. 파라미터 생성

In [10]:
base_data["transdate"] = pd.to_datetime(base_data["transdate"])
start_year = base_data["transdate"].dt.strftime("%Y").min()
end_year = base_data["transdate"].dt.strftime("%Y").max()
city = "울산"

In [11]:
params = parameters(service_key = service_key, 
                    google_key = google_key, 
                    career_net_key = career_net_key, 
                    city = city, 
                    start_year = start_year, 
                    end_year = end_year)

In [12]:
params_dict = params.get()

## 3. 외부 데이터 Load

In [10]:
save_path = "/project/work/data/api_data/"

In [11]:
data_api = Data_Load.Data_API(service_key = service_key, google_key = google_key)

In [12]:
%%time
data_api.get(city = city, params_dict = params_dict)

휴일 정보 수집 ... 완료
날씨 정보 수집 ... n_rows : 900, total_count : 8784, max_page = 10
완료
미세먼지 경보 정보 수집 ... n_rows : 1000, total_count : 304, max_page = 1
완료
상권 정보 수집 ... n_rows : 100, total_count : 36517, max_page = 366
완료
병원 정보 수집 ... n_rows : 1000, total_count : 1406, max_page = 2
완료
학교(초중고) 정보 수집 ... n_rows : 1000, total_count : 500, max_page = 1
완료
대학교 정보 수집 ... 완료
행사 정보 수집 ... n_rows : 1000, total_count : 6396, max_page = 7
완료
축제 정보 수집 ... n_rows : 1000, total_count : 1026, max_page = 2
완료
CPU times: user 26.3 s, sys: 2.11 s, total: 28.4 s
Wall time: 22min 41s


data_api.holiday_data.to_csv('/project/work/data/api_data/holiday_data.csv', index=False)
data_api.weather_data.to_csv('/project/work/data/api_data/weather_data.csv', index=False)
data_api.pm_data.to_csv('/project/work/data/api_data/pm_data.csv', index=False)
data_api.trading_area_data.to_csv('/project/work/data/api_data/trading_area_data.csv', index=False)
data_api.hospital_data.to_csv('/project/work/data/api_data/hospital_data.csv', index=False)
data_api.school_data.to_csv('/project/work/data/api_data/school_data.csv', index=False)
data_api.university_data.to_csv('/project/work/data/api_data/university_data.csv', index=False)
data_api.event_data.to_csv('/project/work/data/api_data/event_data.csv', index=False)
data_api.festival_data.to_csv('/project/work/data/api_data/festival_data.csv', index=False)

In [9]:
with open("data_api_data.pkl", "rb") as f:
    data_api = pickle.load(f)

In [10]:
#with open("data_api_data.pkl", "wb") as f:
#    pickle.dump(data_api, f)

In [23]:
# 휴일 정보 Load
holiday_data = pd.read_csv("/project/work/data/api_data/holiday_data.csv")
# 날씨 정보 Load
weather_data = pd.read_csv("/project/work/data/api_data/weather_data.csv")
# 미세먼지 경보 정보 Load
pm_data = pd.read_csv("/project/work/data/api_data/pm_data.csv")
# 상권 정보 load (기존)
trading_area_data = pd.read_csv("/project/work/data/api_data/trading_area_data.csv")
# 병원 정보 Load
hospital_data = pd.read_csv("/project/work/data/api_data/hospital_data.csv")
# 학교 정보 Load (기존)
school_data = pd.read_csv("/project/work/data/api_data/school_data.csv")
# 대학교 정보 Load (기존)
university_data = pd.read_csv("/project/work/data/api_data/university_data.csv")
# 행사 정보 Load
event_data = pd.read_csv("/project/work/data/api_data/event_data.csv")
# 축제 정보 Load
festival_data = pd.read_csv("/project/work/data/api_data/festival_data.csv")

In [25]:
event_data['eventStartDate'] = pd.to_datetime(event_data['eventStartDate'])

In [26]:
event_data['eventEndDate'] = pd.to_datetime(event_data['eventEndDate'])

In [32]:
event_data['eventStartTime'] = pd.to_datetime(event_data['eventStartTime'])
event_data['eventEndTime'] = pd.to_datetime(event_data['eventEndTime'])

ValueError: ('Unknown string format:', '10:30~11:30')

In [35]:
event_data['eventStartTime'][0]

'16:00'

In [ ]:
eventStartTime

In [14]:
preprocessing = prpr.Preprocess_Data(data = base_data,
                           date_col = "transdate", 
                           target_cols = target_col, 
                           stop_id_col = "stop_id", 
                           holiday_data = holiday_data,
                           weather_data = weather_data,
                           pm_data = pm_data,
                           trading_area_data = trading_area_data,
                           hospital_data = hospital_data,
                           school_data = school_data,
                           university_data = university_data,
                           event_data = event_data,
                           festival_data = festival_data,
                           num_cores = 2)

In [16]:
preprocessing = prpr.Preprocess_Data(data = base_data,
                           date_col = "transdate", 
                           target_cols = target_col, 
                           stop_id_col = "stop_id", 
                           holiday_data = data_api.holiday_data,
                           weather_data = data_api.weather_data,
                           pm_data = data_api.pm_data,
                           trading_area_data = data_api.trading_area_data,
                           hospital_data = data_api.hospital_data,
                           school_data = data_api.school_data,
                           university_data = data_api.university_data,
                           event_data = data_api.event_data,
                           festival_data = data_api.festival_data,
                           num_cores = 2)

In [31]:
event_data.eventStartTime

0            16:00
1            19:30
2            19:30
3            16:00
4            17:30
5            17:30
6            14:00
7            16:30
8            11:00
9            11:00
10           11:00
11           14:00
12           14:00
13           11:00
14           14:00
15           11:00
16           10:30
17           14:00
18            8:00
19     10:30~11:30
20     14:00~16:00
21     14:00~16:00
22     10:00~19:00
23     09:00~18:00
24     10:00~19:00
25     09:00~18:00
26     09:00~18:00
27     11:00~13:00
28     15:00~17:00
29     13:00~15:00
30     11:00~13:00
31     13:00~16:00
32     14:00~16:00
33     10:00~12:00
34     10:30~11:00
35     09:00~18:00
36     09:00~20:00
37     09:00~20:00
38     09:00~18:00
39     00:00~00:00
40     09:30~11:30
41     10:00~12:00
42     10:00~12:00
43     14:00~16:00
44     14:00~17:00
45     10:00~12:00
46     14:00~16:00
47     14:00~16:00
48     16:00~17:30
49     16:00~21:30
50     10:30~12:00
51     10:00~19:00
52     11:00

In [29]:
%%time
ml_data = preprocessing.run()

1. 결측치 Impute ... Finished ((563300, 327))
2. 시계열 변수 생성 : Finished ((563300, 379))
3. 시간적 특성 변수 추가 (특일, 날씨, 미세먼지 경보) ... Finished ((563300, 387))
4. 공간적 특성 정보 추가 (상권정보, 학교정보, 병원정보) ... Finished ((563300, 483))
5. 시공간적 특성 정보 추가 (행사정보, 축제정보) ... 

AttributeError: Can only use .str accessor with string values!

In [ ]:
ml_data

In [18]:
del base_data

## 4. 모델 학습

In [15]:
categorical_feature = ["dayofweek", "hour", "ntl_holi", "holi", "seq_holi"]

In [16]:
numerical_feature = [col for col in ml_data.columns if col not in categorical_feature + [target_col]]

#### 데이터 Setup

In [17]:
reg_experiment = setup(ml_data, 
                       target = "totalcnt", 
                       categorical_features = categorical_feature,
                       numeric_features = numerical_feature,
                       ignore_features = ["month", "weekofyear", "longitude", "latitude", "studentcnt", "childcnt", "normalcnt"],
                       train_size = 0.7,
                       n_jobs = 14,
                       session_id = 12345,
                       silent = True,
                       experiment_name="Passenger_Demand")

,Description,Value
0,session_id,12345
1,Target,totalcnt
2,Original Data,"(563300, 168)"
3,Missing Values,True
4,Numeric Features,155
5,Categorical Features,5
6,Ordinal Features,False
7,High Cardinality Features,False
8,High Cardinality Method,None
9,Transformed Train Set,"(394309, 185)"


#### 초도 모델링

In [22]:
# %%time
# base_models = compare_models(include = ["lightgbm", "rf", "gbr", "lr", "ridge", "lasso", "en", "knn"])

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
lightgbm,Light Gradient Boosting Machine,1.5870,8.2300,2.8686,0.7677,0.5520,0.5649,7.2730
rf,Random Forest Regressor,1.5297,8.3763,2.8939,0.7636,0.5258,0.5834,213.2820
gbr,Gradient Boosting Regressor,1.8527,11.2913,3.3599,0.6814,0.6545,0.5668,64.2760
knn,K Neighbors Regressor,1.7701,11.7880,3.4331,0.6673,0.5823,0.6744,7.6270
lr,Linear Regression,2.0389,12.6811,3.5607,0.6421,0.7164,0.6181,1.4000
ridge,Ridge Regression,2.0389,12.6810,3.5607,0.6421,0.7164,0.6181,0.2960
en,Elastic Net,2.0081,13.2362,3.6378,0.6265,0.7028,0.6053,9.4130
lasso,Lasso Regression,2.0194,13.3677,3.6558,0.6227,0.7072,0.6010,10.9550


CPU times: user 55.6 s, sys: 4.18 s, total: 59.8 s
Wall time: 52min 33s


#### 모델 선택 및 생성

In [23]:
%%time
lightgbm = create_model('lightgbm')

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,1.5685,7.9733,2.8237,0.7685,0.5489,0.5619
1,1.5632,7.9392,2.8177,0.7698,0.5494,0.5624
2,1.6039,8.4129,2.9005,0.7653,0.5561,0.5687
3,1.5805,8.1804,2.8601,0.7634,0.5559,0.5646
4,1.5927,8.3602,2.8914,0.7702,0.5478,0.5591
5,1.5847,8.1915,2.8621,0.7748,0.5484,0.5613
6,1.5860,8.2225,2.8675,0.7666,0.5504,0.5739
7,1.6034,8.5396,2.9223,0.7617,0.5526,0.5698
8,1.6016,8.4452,2.9061,0.7667,0.5544,0.5622
9,1.5860,8.0354,2.8347,0.7701,0.5558,0.5652


CPU times: user 48.3 s, sys: 1.7 s, total: 50 s
Wall time: 1min 13s


#### 모델 튜닝

In [24]:
%%time
tuned_lightgbm_optuna_tpe = tune_model(lightgbm, 
                                       optimize = "RMSE", 
                                       search_library = "optuna", 
                                       search_algorithm = "tpe", 
                                       fold = 10, 
                                       return_tuner = True, 
                                       n_iter = 10)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,1.4772,7.3861,2.7177,0.7856,0.5034,0.5604
1,1.4741,7.3238,2.7062,0.7876,0.5068,0.5597
2,1.5056,7.6849,2.7722,0.7856,0.5082,0.5649
3,1.4842,7.4553,2.7304,0.7844,0.5103,0.5574
4,1.5085,7.7432,2.7827,0.7872,0.5071,0.5521
5,1.5008,7.6807,2.7714,0.7888,0.5055,0.5553
6,1.4879,7.4835,2.7356,0.7876,0.5057,0.5665
7,1.5139,7.9616,2.8216,0.7779,0.5088,0.5664
8,1.5234,7.8637,2.8042,0.7827,0.5160,0.5578
9,1.4967,7.4046,2.7211,0.7882,0.5099,0.5625


CPU times: user 2h 56min 31s, sys: 4h 2min 9s, total: 6h 58min 41s
Wall time: 29min 4s


In [27]:
with open("tuned_lightgbm_optuna_tpe.pkl", "wb") as f:
    pickle.dump(tuned_lightgbm_optuna_tpe, f)

In [31]:
%%time
ridge = create_model('ridge')

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,2.0088,12.0995,3.4784,0.6487,0.7117,0.6188
1,2.0121,12.2983,3.5069,0.6434,0.7151,0.6154
2,2.0613,13.1093,3.6207,0.6343,0.7182,0.6231
3,2.0316,12.5272,3.5394,0.6377,0.7206,0.6159
4,2.0469,12.8925,3.5906,0.6456,0.7113,0.6103
5,2.0489,12.8109,3.5792,0.6477,0.7155,0.6096
6,2.0284,12.4503,3.5285,0.6466,0.7132,0.6263
7,2.0534,12.9802,3.6028,0.6378,0.7185,0.6240
8,2.0613,13.1608,3.6278,0.6364,0.7196,0.6167
9,2.0366,12.4806,3.5328,0.6430,0.7204,0.6213


CPU times: user 1.04 s, sys: 4.25 s, total: 5.29 s
Wall time: 4.2 s


In [32]:
%%time
tuned_ridge_optuna_tpe = tune_model(ridge, 
                                       optimize = "RMSE", 
                                       search_library = "optuna", 
                                       search_algorithm = "tpe", 
                                       fold = 10, 
                                       return_tuner = True, 
                                       n_iter = 10)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,2.0087,12.0995,3.4784,0.6487,0.7116,0.6188
1,2.0120,12.2981,3.5069,0.6434,0.7151,0.6154
2,2.0613,13.1092,3.6207,0.6343,0.7182,0.6231
3,2.0316,12.5270,3.5393,0.6377,0.7206,0.6159
4,2.0469,12.8923,3.5906,0.6456,0.7112,0.6103
5,2.0489,12.8109,3.5792,0.6477,0.7155,0.6096
6,2.0284,12.4502,3.5285,0.6466,0.7132,0.6262
7,2.0533,12.9800,3.6028,0.6378,0.7185,0.6240
8,2.0613,13.1610,3.6278,0.6364,0.7196,0.6167
9,2.0365,12.4805,3.5328,0.6430,0.7203,0.6212


CPU times: user 2min 1s, sys: 5min 27s, total: 7min 28s
Wall time: 45.7 s


In [33]:
with open("tuned_ridge_optuna_tpe.pkl", "wb") as f:
    pickle.dump(tuned_ridge_optuna_tpe, f)

In [34]:
stacked = stack_models([tuned_lightgbm_optuna_tpe[0], tuned_ridge_optuna_tpe[0]])

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,1.4829,7.3841,2.7174,0.7856,0.5033,0.5635
1,1.4775,7.2904,2.7001,0.7886,0.5061,0.5615
2,1.5077,7.6460,2.7651,0.7867,0.5078,0.5664
3,1.4864,7.4170,2.7234,0.7855,0.5088,0.5592
4,1.5107,7.7312,2.7805,0.7875,0.5059,0.5537
5,1.5067,7.6834,2.7719,0.7887,0.5061,0.5591
6,1.4904,7.4512,2.7297,0.7885,0.5050,0.5682
7,1.5167,7.9410,2.8180,0.7784,0.5079,0.5682
8,1.5252,7.8325,2.7987,0.7836,0.5139,0.5595
9,1.5007,7.4051,2.7212,0.7882,0.5090,0.5658


#### 모델 평가

In [35]:
%%time
evaluate_model(stacked)

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Hyperparameters', 'param…

CPU times: user 672 ms, sys: 2.5 s, total: 3.17 s
Wall time: 270 ms


In [ ]:
test_tuned_lightgbm_optuna_tpe